<a href="https://colab.research.google.com/github/rara-by/Practice-Notebooks/blob/main/DSCI6004_Fine_tune_for_Cybersecurity_NER_Sharara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Transformers installation
! pip install transformers datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00


# Text classification

In [ ]:
!pip install -U accelerate
!pip install -U transformers

# then restart runtime to T4 GPU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 7.0 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00


## Load dataset

In [ ]:
label2id = {'I-Malware': 0,
 'I-Organization': 1,
 'B-Indicator': 2,
 'I-Vulnerability': 3,
 'I-System': 4,
 'O': 5,
 'B-Vulnerability': 6,
 'B-Organization': 7,
 'B-Malware': 8,
 'B-System': 9,
 'I-Indicator': 10}

In [ ]:
import pandas as pd

df = pd.read_csv("train.txt", delimiter="\t")
df = df.dropna()
for idx, row in enumerate(df.iloc[:, 1]):
  if row in label2id:
    df.iloc[idx, 1] = label2id[row]

print(df.head())
df.to_csv("train.txt", index=False, header=False, sep="\t")

df = pd.read_csv("valid.txt", delimiter="\t")
df = df.dropna()
for idx, row in enumerate(df.iloc[:, 1]):
  if row in label2id:
    df.iloc[idx, 1] = label2id[row]
df.to_csv("valid.txt", index=False, header=False, sep="\t")

df = pd.read_csv("test.txt", delimiter="\t")
df = df.dropna()
for idx, row in enumerate(df.iloc[:, 1]):
  if row in label2id:
    df.iloc[idx, 1] = label2id[row]
df.to_csv("test.txt", index=False, header=False, sep="\t")

     Super B-Malware
0    Mario         0
1      Run         0
2  Malware         0
3        #         5
4        2         5


In [ ]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files={"train": "train.txt", "valid": "valid.txt",
                                           "test": "test.txt"},  delimiter='\t',
                       column_names=["text", "label"])

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Then take a look at an example:

In [ ]:
dataset["test"][0]

{'text': 'Did', 'label': 5}

There are two fields in this dataset:

- `text`: the movie review text.
- `label`: a value that is either `0` for a negative review or `1` for a positive review.

## Preprocess

The next step is to load a DistilBERT tokenizer to preprocess the `text` field:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", is_split_into_words=True)

Now create a batch of examples using [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/26832 [00:00<?, ? examples/s]

Map:   0%|          | 0/19529 [00:00<?, ? examples/s]

Map:   0%|          | 0/6993 [00:00<?, ? examples/s]

## Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [accuracy](https://huggingface.co/spaces/evaluate-metric/accuracy) metric (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric):

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the accuracy:

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Train

Before you start training your model, create a map of the expected ids to their labels with `id2label` and `label2id`:

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 26832
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 19529
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 6993
    })
})

In [ ]:
dataset["test"].features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None)}

In [ ]:
id2label = {v: k for k, v in label2id.items()}
id2label

{0: 'I-Malware',
 1: 'I-Organization',
 2: 'B-Indicator',
 3: 'I-Vulnerability',
 4: 'I-System',
 5: 'O',
 6: 'B-Vulnerability',
 7: 'B-Organization',
 8: 'B-Malware',
 9: 'B-System',
 10: 'I-Indicator'}

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load DistilBERT with [AutoModelForSequenceClassification](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForSequenceClassification) along with the number of expected labels, and the label mappings:

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=11, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


At this point, only three steps remain:

1. Define your training hyperparameters in [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. You'll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) will evaluate the accuracy and save the training checkpoint.
2. Pass the training arguments to [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [ ]:
training_args = TrainingArguments(
    output_dir="NER_classification",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False, # do i push the weights to my hugging face hub?
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.143600,0.204009,0.957243
2,0.108800,0.188678,0.960725


TrainOutput(global_step=3354, training_loss=0.15439984396091158, metrics={'train_runtime': 2436.7506, 'train_samples_per_second': 22.023, 'train_steps_per_second': 1.376, 'total_flos': 7107412127772096.0, 'train_loss': 0.15439984396091158, 'epoch': 2.0})

## Inference

Great, now that you've finetuned a model, you can use it for inference!

Grab some text you'd like to run inference on:

In [ ]:
text = "devices"

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for sentiment analysis with your model, and pass your text to it:

In [ ]:
from transformers import pipeline

classifier = pipeline("token-classification", model="/content/NER_classification/checkpoint-3354")
classifier(text)

[{'entity': 'I-Indicator',
  'score': 0.16991566,
  'index': 1,
  'word': 'mal',
  'start': 0,
  'end': 3},
 {'entity': 'I-Indicator',
  'score': 0.19113678,
  'index': 2,
  'word': '##ware',
  'start': 3,
  'end': 7}]

You can also manually replicate the results of the `pipeline` if you'd like:

Tokenize the text and return PyTorch tensors:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/content/NER_classification/checkpoint-3354")
inputs = tokenizer(text, return_tensors="pt")

Pass your inputs to the model and return the `logits`:

In [ ]:
from transformers import AutoModelForSequenceClassification
import torch
model = AutoModelForSequenceClassification.from_pretrained("/content/NER_classification/checkpoint-3354")
with torch.no_grad():
    logits = model(**inputs).logits

Get the class with the highest probability, and use the model's `id2label` mapping to convert it to a text label:

In [ ]:
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'O'